In [ ]:
import intake
import xarray as xr
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
from dask.distributed import Client
client = Client(threads_per_worker=1)
client

In [ ]:
%%time
catalog_path = '/g/data/v14/tm4888/code/BRAN2020-intake-catalog/catalogs/'
BRAN2020_catalog = intake.open_esm_datastore(catalog_path+'BRAN2020.json', columns_with_iterables=['variable']) # load catalogue
search_uv_month = BRAN2020_catalog.search(variable=['u','v'],time_period='month') # search and filter data by variables and time period
DS = search_uv_month.to_dask() # load data lazily with Dask
DS_slice = DS.sel(st_ocean= slice(0,300)).sel(xu_ocean=slice(130,250)).sel(yu_ocean=slice(-30,30)).mean('st_ocean') # slice out reduced XYZ subset required
clim_uv = DS_slice.groupby('Time.month').mean(method="cohorts", engine="flox") ## calculate climatology & chunking rules everything!
speed = np.sqrt(clim_uv.u**2 + clim_uv.v**2) # calculate current speeds
speed.sel(month=8).plot(robust=True) #plot
plt.title('BRAN2020 current speed\n August climatology')

# compute uv climatology over upper 300m 

In [ ]:
%%time
clim_uv = clim_uv.compute()

In [ ]:
%%time
speed = speed.compute()

# write out to scratch

In [ ]:
PICOSOM_BRAN2020_uv_speed = xr.merge([speed.rename('speed'),clim_uv])

In [ ]:
PICOSOM_BRAN2020_uv_speed

In [ ]:
%%time
PICOSOM_BRAN2020_uv_speed.to_netcdf('/scratch/xv83/tm4888/PICOSOM_BRAN2020/PICOSOM_BRAN2020_uv_speed.nc')

# read in from nc file

In [ ]:
PICOSOM_BRAN2020_uv_speed_ = xr.open_dataset('/scratch/xv83/tm4888/PICOSOM_BRAN2020/PICOSOM_BRAN2020_uv_speed.nc')

In [ ]:
PICOSOM_BRAN2020_uv_speed_

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
import matplotlib.ticker as ticker
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy
from matplotlib import mlab, cm, gridspec
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
%matplotlib inline 

In [ ]:
%%time
for time_choice in range(1,13):
    u = PICOSOM_BRAN2020_uv_speed_.u.sel(month=time_choice)
    v = PICOSOM_BRAN2020_uv_speed_.v.sel(month=time_choice)
    speed = PICOSOM_BRAN2020_uv_speed_.speed.sel(month=time_choice)

    #plot model data
    transform = ccrs.PlateCarree()
    cmap = 'Spectral_r'
    cbar_label='current speed'
    plot_data = speed

    ###
    fig = plt.figure(num=None, figsize=(16, 9), dpi=600, facecolor='w', edgecolor='k')
    ax = plt.subplot(projection=ccrs.PlateCarree(180))
    ax.set_extent([130,250, -30, 30], ccrs.PlateCarree())
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '50m', edgecolor='face', facecolor='white'))
    ax.coastlines('50m',linewidth=0.5,edgecolor='grey')
    plot_data.plot(transform=transform,cmap=cmap,robust=True,add_colorbar=False)

    #plot u/v vectors
    # Define the x and y coordinates
    x = PICOSOM_BRAN2020_uv_speed_.xu_ocean
    y = PICOSOM_BRAN2020_uv_speed_.yu_ocean
    ax.quiver(x.values,y.values,u.values,v.values,transform=transform, units='x', width=0.01, scale=0.7, headwidth=2,alpha=0.2)
    #ax.set_title('BRAN2020 1993-2022\ncurrent speed \n August Climatology')
    plot_DIR = '/scratch/xv83/tm4888/PICOSOM_BRAN2020/'
    plot_file_name = 'PICOSOM_BRAN2020_'+str(time_choice)+'.png'
    plot_path = plot_DIR + plot_file_name
    plt.savefig(plot_path, dpi = 600, bbox_inches='tight')